The reference Collab Guide: https://colab.research.google.com/drive/1n5yCFsn9a76rNDx6dnwgdFLLoUIZATSS#scrollTo=8TRAh8G6sNfA

The reference repositories: 
Original: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=Ylscg48YYxfF

My copy: https://github.com/huggingface/diffusers/tree/main#new--stable-diffusion-is-now-fully-compatible-with-diffusers

The image editing library: https://pillow.readthedocs.io/en/stable/index.html

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.9.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install accelerate

In [24]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
pipe = pipe.to("cuda")

#prompt = "a photo of an astronaut riding a horse on mars"
#image = pipe(prompt).images[0]  
    
#image.save("astronaut_rides_horse.png")

In [27]:
from PIL import Image
from torch import autocast
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_cols = 2
num_rows = 2

prompt = ["an image of a person enjoying a refreshing drink from a durable, high-quality water bottle. The bottle should be prominently featured, with a clear and attractive design. The background should be a vibrant outdoor setting, and the image should be in 4k resolution for optimal printing and display."] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

In [ ]:
#Print one of the 4 images seperately
with grid as im:

    # The crop method from the Image module takes four coordinates as input.
    # The right can also be represented as (left+width)
    # and lower can be represented as (upper+height).
    (left1, upper1, right1, lower1) = (0, 0, 512, 512)
    (left2, upper2, right2, lower2) = (512,0,1024,512)
    (left3, upper3, right3, lower3) = (0,512,512,1024)
    (left4, upper4, right4, lower4) = (512,512,1024,1024)

im_first = im.crop((left1, upper1, right1, lower1))
im_second = im.crop((left2, upper2, right2, lower2))
im_third = im.crop((left3, upper3, right3, lower3))
im_fourth = im.crop((left4, upper4, right4, lower4))

#print = im_first
#print = im_second
#print = im_third
print = im_fourth
print